### Imports e conexão

In [72]:
import pyodbc
import pandas as pd

In [73]:
server = 'localhost'  # ou o endereço IP do servidor SQL Server
database = 'Neteye'   # nome do banco de dados restaurado
username = 'sa'       # nome de usuário do SQL Server
password = 'Morg@1812'  # senha do usuário


# String de conexão
connection_string = (
    f'DRIVER={{ODBC Driver 18 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password};'
    'TrustServerCertificate=yes;'
)

try:
    # Estabelecer conexão com o banco de dados
    connection = pyodbc.connect(connection_string)
    print("Conexão bem-sucedida!")

except Exception as e:
    print("Erro ao conectar ao banco de dados:", e)

Conexão bem-sucedida!


In [74]:
query = """SELECT * FROM VW_PROGRAMS"""
vw_programs = pd.read_sql_query(query, connection)

query = """SELECT * FROM REGRASCLASSIFICACAO"""
regras_class = pd.read_sql_query(query, connection)

query = """SELECT * FROM CLASSIFICACAOSOFTWARE"""
class_soft = pd.read_sql_query(query, connection)

query = """SELECT * FROM bi.fPRODUTIVIDADE"""
bi_prod = pd.read_sql_query(query, connection)

query = """SELECT * FROM bi.fSESSOES"""
bi_sessoes = pd.read_sql_query(query, connection)

query = """SELECT * FROM USUARIOS"""
usuarios = pd.read_sql_query(query, connection)

query = """SELECT * FROM SETORES"""
setores = pd.read_sql_query(query, connection)

/tmp/ipykernel_83024/1828477103.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vw_programs = pd.read_sql_query(query, connection)
/tmp/ipykernel_83024/1828477103.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  regras_class = pd.read_sql_query(query, connection)
/tmp/ipykernel_83024/1828477103.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  class_soft = pd.read_sql_query(query, connection)
/tmp/ipykernel_83024/1828477103.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or datab

#### Análise para usuário de SQ 367

In [75]:
usr = "USUARIO_467"

De que setor ele é?

In [76]:
usuarios.head(1)

,CODCONSOLE,USUARIO,SETOR_ID,ADDINFO1,ADDINFO2,ADDINFO3,ADDINFO4,ADDINFO5,MONITOR,ATIVO,NOME
0,EMPRESA,USUARIO_001,18.0,None,None,None,None,None,3,1,Usuário 001 da Silva


In [77]:
usuarios[usuarios['USUARIO'] == usr]

,CODCONSOLE,USUARIO,SETOR_ID,ADDINFO1,ADDINFO2,ADDINFO3,ADDINFO4,ADDINFO5,MONITOR,ATIVO,NOME


In [78]:
setores.head(2)

,SETOR_ID,CODCONSOLE,SETOR,RESPONSABLE,RESP_MAIL,FONE,AVG_SALARY,MONITOR,LDAP_SETOR,LDAP_SETOR_DN,seq
0,1,EMPRESA,TECNOLOGIA DA INFORMAÇÃO,,,,None,0,None,None,35
1,2,EMPRESA,SUPRIMENTOS,,,,None,0,None,None,34


Como é um dia de trabalho desse cara?

In [79]:
bi_prod.SQ_USUARIO.value_counts()

SQ_USUARIO
324    467
328    431
313    349
385    332
311    307
      ... 
70       2
367      1
85       1
46       1
19       1
Name: count, Length: 117, dtype: int64

In [89]:
bi_sessoes[bi_sessoes['ID_USUARIO'] == 'USUARIO_367']

,DT_SESSAO,SQ_USUARIO,ID_USUARIO,LOGIN,LOGOUT,QT_SEGUNDOS,DIA_SEMANA,HORAS
0,2024-08-16,13,USUARIO_367,2024-08-16 18:33:23.523,2024-08-16 22:17:53,13470,Sexta-feira,3.74
100,2024-08-17,13,USUARIO_367,2024-08-17 07:56:25.000,2024-08-17 11:38:03,13298,Sábado,3.69


Adicionando coluna com referencia de dia da semana e de horas trabalhadas.

In [81]:
dias_semana_pt = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

In [82]:
days_map = {
    'Monday': 'Segunda-feira', 'Tuesday': 'Terça-feira', 'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira', 'Friday': 'Sexta-feira', 'Saturday': 'Sábado', 'Sunday': 'Domingo'
}

bi_sessoes['DT_SESSAO'] = pd.to_datetime(bi_sessoes['DT_SESSAO'])
bi_sessoes['DIA_SEMANA'] = bi_sessoes['DT_SESSAO'].dt.day_name().map(days_map)

In [83]:
bi_sessoes['HORAS'] = (bi_sessoes['QT_SEGUNDOS'] // 3600 + (bi_sessoes['QT_SEGUNDOS'] % 3600 / 60) / 60).round(2)

In [84]:
bi_sessoes[bi_sessoes['ID_USUARIO'] == usr]

,DT_SESSAO,SQ_USUARIO,ID_USUARIO,LOGIN,LOGOUT,QT_SEGUNDOS,DIA_SEMANA,HORAS


Vamos analisar o dia de sexta-feira.

In [85]:
filtered_df = vw_programs[(vw_programs['LOGIN'] == 'USUARIO_367') & (vw_programs['INITIALDATE'].dt.date == pd.Timestamp('2024-08-16').date())]

In [86]:
regras_class.rename(columns={"ID":"REGRA_ID"}, inplace=True)

In [87]:
filtered_df = filtered_df.merge(on="REGRA_ID", right=regras_class[["REGRA_ID", "ID_CLASSIFICACAO"]], how="left").merge(left_on="ID_CLASSIFICACAO", right=class_soft, right_on="ID", how="left")

In [88]:
filtered_df

,ID_x,CODCONSOLE,STATION,HANDLE,ISOPEN,ACTPROGRAM,IP,URL,INITIALDATE,ACTPROC,LOGIN,FINALDATE,COUNTMIN,REMOTEPORT,REGRA_ID,ORIGEM_HORA,ID_CLASSIFICACAO,ID_y,CLASSIFICACAO,PRODUCTIVITY_LEVEL
0,2920,EMPRESA,ESTACAO187,395784,0,,187.109.98.167,None,2024-08-16 18:32:55.740,EXPLORER.EXE,USUARIO_367,2024-08-16 21:09:50.000,0.000000,52183,18.0,0,23.0,23.0,Sistema Operacional,1.0
1,2928,EMPRESA,ESTACAO187,65936,0,Program Manager,187.109.98.167,,2024-08-16 18:33:23.523,EXPLORER.EXE,USUARIO_367,2024-08-16 21:09:50.000,0.116667,52183,18.0,0,23.0,23.0,Sistema Operacional,1.0
2,2929,EMPRESA,ESTACAO187,1313250,0,,187.109.98.167,None,2024-08-16 18:33:23.990,EXPLORER.EXE,USUARIO_367,2024-08-16 21:09:50.000,0.000000,52183,18.0,0,23.0,23.0,Sistema Operacional,1.0
3,3782,EMPRESA,ESTACAO187,68522,0,Alternância de Tarefas,187.109.98.167,,2024-08-16 20:59:12.000,EXPLORER.EXE,USUARIO_367,2024-08-16 21:09:50.000,0.266667,52183,18.0,0,23.0,23.0,Sistema Operacional,1.0
4,3823,EMPRESA,ESTACAO187,67086,0,Widgets,187.109.98.167,,2024-08-16 21:31:33.000,MSEDGEWEBVIEW2.EXE,USUARIO_367,2024-08-16 21:31:49.000,0.000000,52174,61.0,0,8.0,8.0,Corporativo,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,3239,EMPRESA,ESTACAO187,657506,0,Lixo Eletrônico - THIAGO@***.com.BR - Outlook,187.109.98.167,,2024-08-16 19:00:11.000,OUTLOOK.EXE,USUARIO_367,2024-08-16 19:02:06.000,0.283333,52183,11.0,0,11.0,11.0,Email,0.0
159,3884,EMPRESA,ESTACAO187,526246,0,#chaos-shadow-bosses | VanHorn Gaming - Discord,187.109.98.167,,2024-08-16 21:55:53.000,DISCORD.EXE,USUARIO_367,2024-08-16 21:56:09.000,0.266667,52174,NaN,0,NaN,NaN,NaN,NaN
160,3885,EMPRESA,ESTACAO187,526246,0,#??general-chat | VanHorn Gaming - Discord,187.109.98.167,,2024-08-16 21:56:09.000,DISCORD.EXE,USUARIO_367,2024-08-16 22:00:07.000,0.000000,52174,NaN,0,NaN,NaN,NaN,NaN
161,3266,EMPRESA,ESTACAO187,657506,0,Caixa de Entrada - THIAGO@***.com.BR - Outlook,187.109.98.167,,2024-08-16 19:02:06.000,OUTLOOK.EXE,USUARIO_367,2024-08-16 19:02:20.000,0.266667,52183,11.0,0,11.0,11.0,Email,0.0


In [90]:
class_soft

,ID,CLASSIFICACAO,PRODUCTIVITY_LEVEL
0,1,Armazenamento em nuvem,1
1,2,Arquivo,1
2,3,Bancos,0
3,4,Blogs,-1
4,5,Compras On-line,-1
5,6,Comunicação,0
6,7,Conteúdo Adulto,-1
7,8,Corporativo,1
8,9,Desenvolvimento,1
9,10,Educação,0
